<h2>import data from excel</h2>

In [14]:
from gurobipy import*
import math
import pandas as pd
import numpy as np
import csv

In [15]:
df={}
Demand_num=1000 #follow the input file
Demand = list()
Demand = list(range(1,Demand_num+1))
df=pd.read_csv("./prj1_test_data/Car_Demand_"+str(Demand_num)+".csv")
Type_d={}
Location_d={}
Time={}
Penalty={}
for i in range(1,Demand_num+1):
    Location_d[i] = list()
    Time[i] = list()
    Type_d[i]=df.iloc[i-1]["type"]
    Location_d[i].append(df.iloc[i-1]["Location_X"])
    Location_d[i].append(df.iloc[i-1]["Location_Y"])
    Time[i].append(df.iloc[i-1]["Time_H"])
    Time[i].append(df.iloc[i-1]["Time_E"])
    Penalty[i]=df.iloc[i-1]["Penalty"]


In [16]:
# print(Demand)
# print(Type_d)
# print(Location_d)
# print(Time)
# print(Penalty)

In [17]:
df2={}
Supply_num=200
Supply = list()
Supply = list(range(1,Supply_num+1))
df2=pd.read_csv("./prj1_test_data/Space_"+str(Supply_num)+".csv")
Type_s={}
Location_s={}
for i in range(1,Supply_num+1):
    Location_s[i]=list()
    Type_s[i]=df2.iloc[i-1]["type"]
    Location_s[i].append(df2.iloc[i-1]["Location_X"])
    Location_s[i].append(df2.iloc[i-1]["Location_Y"])

In [18]:
# print(Supply)
# print(Type_s)
# print(Location_s)

In [19]:
def check_time_overlap(a,b):
    if (a[0]<b[0]):
        if (b[0]-a[0]) < (a[1] - a[0]):
            return True
        else:
            return False
    elif (a[0]>b[0]):
        if (a[0]-b[0]) < (b[1] - b[0]):
            return True
        else:
            return False
    else :
        return True
        
def distance(a,b):
    return math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)


In [20]:
# print(check_time_overlap([100,150],[0,100]))
# print(distance([45,8],[30,33]))

In [21]:
model = Model("Car_parking")
x={}
y={}
b={}

<h2>init var</h2>

In [22]:
for d in Demand:
    y[d] = model.addVar(vtype = "B",name = "y(%s)"%d)
    for s in Supply:
        x[d,s] = model.addVar(vtype="B",name = "x(%s,%s)"%(d,s))
        b[d,s] = model.addVar(lb = 0, vtype="C", name = "b(%s,%s)"%(d,s))
model.update()

<h1>add constraint</h1>
<h2>time constraint</h2>
<br>
<h3>one demand correspond to one supply</h3>
<h3>But a supply can correspond to several demand</h3>
<p>quicksum(x[d,s] for s in Supply) = 1 for d in Demand</p>
<br>
<h3>Type constraint</h3>
   

In [23]:
R=20
# #demand constraint one car can at most park in one lot
# for d in Demand:
#     model.addConstr(quicksum(x[d,s] for s in Supply) <= 1,name = "Demand constraint")
    
#time constraint
# for i in Demand:
#     for j in range(i+1,Demand_num+1):
#         if check_time_overlap(Time[i],Time[j]) == True:
#             for s in Supply:
#                 model.addConstr((x[i,s]+x[j,s]) <= 1, name = "Time overlap constraint" ) 
                
# # type constraint
# for d in Demand:
#     for s in Supply:
#         if Type_d[d] > Type_s[s]:
#             model.addConstr(x[d,s] == 0, name = "Type constraint")
            
# # distance constriant 
# for d in Demand:
#     for s in Supply:
#         if distance(Location_d[d],Location_s[s]) > 20:
#             model.addConstr(x[d,s] == 0, name = "distance constraint")

# # penalty contraint sum(x[d,s] for Supply)+y ==1
# for d in Demand:
#     model.addConstr(quicksum(x[d,s] for s in Supply) + y[d] == 1, name = "penalty constraint")

# # benefit constraint 
# for d in Demand:
#     for s in Supply:
#         radius = distance(Location_d[d],Location_s[s])
#         model.addConstr(b[d,s] == (Penalty[d]/radius), name = "benefit define")

#combine all except time
# for d in Demand:
#     model.addConstr(quicksum(x[d,s] for s in Supply) <= 1,name = "Demand constraint")
#     model.addConstr(quicksum(x[d,s] for s in Supply) + y[d] == 1, name = "penalty constraint")
#     for s in Supply:
#         if Type_d[d] > Type_s[s]:
#             model.addConstr(x[d,s] == 0, name = "Type constraint")
#             continue
#         if distance(Location_d[d],Location_s[s]) > 20:
#             model.addConstr(x[d,s] == 0, name = "distance constraint")
#             continue
#         radius = distance(Location_d[d],Location_s[s])
#         model.addConstr(b[d,s] == (Penalty[d]/radius), name = "benefit define")

vari={}
for d in Demand:
    model.addConstr(quicksum(x[d,s] for s in Supply) <= 1,name = "Demand constraint")
    model.addConstr(quicksum(x[d,s] for s in Supply) + y[d] == 1, name = "penalty constraint")
    for s in Supply:
        if Type_d[d] > Type_s[s]:
            model.addConstr(x[d,s] == 0, name = "Type constraint")
            vari[d,s]=0
            continue
        if distance(Location_d[d],Location_s[s]) > R:
            model.addConstr(x[d,s] == 0, name = "distance constraint")
            vari[d,s]=0
            continue
        radius = distance(Location_d[d],Location_s[s])
        vari[d,s]=1 #means the car is suitable for the supply
        model.addConstr(b[d,s] == (Penalty[d]/radius), name = "benefit define")
        
for i in Demand:
    for j in range(i+1,Demand_num+1):
        if check_time_overlap(Time[i],Time[j]) == True:
            for s in Supply:
                if vari[i,s] == 0 or vari[j,s] == 0:
                    continue
                model.addConstr((x[i,s]+x[j,s]) <= 1, name = "Time overlap constraint" ) 
        
            



In [24]:

model.setObjective(quicksum(b[d,s]*x[d,s] for (d,s) in x)-quicksum(Penalty[d]*y[d] for d in Demand),GRB.MAXIMIZE)

In [25]:
model.optimize()
print("optimize",model.objVal)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5111418 rows, 401000 columns and 10419836 nonzeros
Model fingerprint: 0x759702fb
Model has 200000 quadratic objective terms
Variable types: 200000 continuous, 201000 integer (201000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 5e+02]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -279451.0000
Presolve removed 200000 rows and 355851 columns (presolve time = 6s) ...
Presolve removed 201000 rows and 355855 columns (presolve time = 12s) ...
Presolve removed 201000 rows and 355903 columns (presolve time = 15s) ...
Presolve removed 5028355 rows and 355903 columns (presolve time = 20s) ...
Presolve removed 5037426 rows and 355903 columns (presolve time = 25s) ...
Presolve removed 5042520 rows and 355903 co

Thread count was 8 (of 8 available processors)

Solution count 10: -39931.2 -40737.9 -47510.8 ... -278990
No other solutions better than -39931.2

Optimal solution found (tolerance 1.00e-04)
Best objective -3.993118496360e+04, best bound -3.993118496360e+04, gap 0.0000%
optimize -39931.18496360088


In [60]:

for d in Demand:
    for s in Supply:
        if x[d,s].X > 0:
            print("car %s park in space %s: %s"%(d,s,x[d,s].X))

car 1 park in space 39: 1.0
car 2 park in space 175: 1.0
car 4 park in space 98: 1.0
car 6 park in space 109: 1.0
car 7 park in space 7: 1.0
car 8 park in space 194: 1.0
car 11 park in space 119: 1.0
car 13 park in space 160: 1.0
car 14 park in space 141: 1.0
car 15 park in space 76: 1.0
car 17 park in space 127: 1.0
car 22 park in space 134: 1.0
car 23 park in space 126: 1.0
car 28 park in space 16: 1.0
car 29 park in space 66: 1.0
car 32 park in space 96: 1.0
car 33 park in space 31: 1.0
car 34 park in space 102: 1.0
car 35 park in space 169: 1.0
car 36 park in space 17: 1.0
car 37 park in space 133: 1.0
car 38 park in space 12: 1.0
car 41 park in space 143: 1.0
car 42 park in space 52: 1.0
car 44 park in space 18: 1.0
car 48 park in space 120: 1.0
car 49 park in space 174: 1.0
car 52 park in space 169: 1.0
car 55 park in space 131: 1.0
car 56 park in space 192: 1.0
car 58 park in space 64: 1.0
car 62 park in space 177: 1.0
car 64 park in space 62: 1.0
car 67 park in space 147: 1.0
c

car 619 park in space 137: 1.0
car 622 park in space 193: 1.0
car 623 park in space 136: 1.0
car 624 park in space 14: 1.0
car 625 park in space 108: 1.0
car 632 park in space 123: 1.0
car 635 park in space 53: 1.0
car 637 park in space 158: 1.0
car 638 park in space 165: 1.0
car 639 park in space 161: 1.0
car 640 park in space 46: 1.0
car 641 park in space 139: 1.0
car 643 park in space 141: 1.0
car 644 park in space 43: 1.0
car 646 park in space 145: 1.0
car 650 park in space 136: 1.0
car 653 park in space 171: 1.0
car 655 park in space 119: 1.0
car 656 park in space 113: 1.0
car 657 park in space 69: 1.0
car 659 park in space 19: 1.0
car 660 park in space 2: 1.0
car 662 park in space 14: 1.0
car 663 park in space 110: 1.0
car 665 park in space 134: 1.0
car 666 park in space 154: 1.0
car 667 park in space 86: 1.0
car 669 park in space 168: 1.0
car 670 park in space 75: 1.0
car 671 park in space 113: 1.0
car 673 park in space 172: 1.0
car 674 park in space 44: 1.0
car 678 park in spac